In [18]:
import os
import shutil
import random
import json
from midi2audio import FluidSynth

import numpy as np
import tensorflow as tf
from tensorflow import keras

import pydub
from pydub import AudioSegment

import librosa
import random

In [35]:
# this function will print the output of cmd ON TIME
# ref: https://lumeng.blog.csdn.net/article/details/104845611
def runCMD(cmd):
    screenData = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    while True:
        line = screenData.stdout.readline()
        print(line.decode('gbk').strip("b'"))
        if line == b'' or subprocess.Popen.poll(screenData) == 0:
            screenData.stdout.close()
            break

In [9]:
# dir
config = r'cat-mel_2bar_small'
run_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100'+'\\'+config+'\\'
train_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\train'
checkpoint_file = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\cat-mel_2bar_small\train'
interMusic_1 = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\59.mp3.mid'
interMusic_2 = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\89.mp3.mid'
output_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output'

In [37]:
# interpolate

music_interpolate = r'music_vae_generate \
--config={config} \
--checkpoint_file={checkpoint_file} \
--mode=interpolate \
--num_outputs={num_outputs} \
--input_midi_1={interMusic_1} \
--input_midi_2={interMusic_2} \
--output_dir={output_dir}'

runCMD(music_interpolate.
          format(config=config,checkpoint_file=checkpoint_file,num_outputs=5,
                interMusic_1=interMusic_1, interMusic_2=interMusic_2,output_dir=output_dir))


Instructions for updating:

non-resource variables are not supported in the long term

INFO:tensorflow:Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

I1102 16:28:25.919774 26644 music_vae_generate.py:145] Attempting to extract examples from input MIDIs using config `cat-mel_2bar_small`...

5 valid inputs extracted from `C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\59.mp3.mid`. Outputting these potential inputs as `C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-11-02_162825-*-of-005.mid`. Call script again with one of these instead.




In [ ]:
audio-to-midi

In [11]:
# covert output.midi into wav file
for parent, dirnames, filenames in os.walk(output_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mid"):
            index = os.path.splitext(filename)[0]
            print(os.path.join(output_dir, filename))
            FluidSynth().midi_to_audio(os.path.join(output_dir, filename),os.path.join(output_dir,index+'.wav'))

C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-10-16_010019-000-of-005.mid


FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [ ]:
# load best model
model = keras.models.load_model(r'C:\Users\Arnold\Desktop\4560\CNN\best_crnn')

In [ ]:
# run the model on the output.wav
for parent, dirnames, filenames in os.walk(output_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".wav"):
            index = os.path.splitext(filename)[0]
            print(os.path.join(output_dir, filename))
            wavsong = AudioSegment.from_wav(os.path.join(output_dir, filename))
            
            one_data = []
            test_data = []

            # mel freq
            for i in range(4):
                outputpath = os.path.join(output_dir,index+'-'+str(i)+".wav")
                wavsong[(500*i):(500+500*i)].export(outputpath, format="wav")
                y, sr = librosa.load(outputpath, sr=None)
                os.remove(outputpath)

                fragment = np.array(
                                np.log(
                                    librosa.feature.melspectrogram(
                                        y, sr, n_mels=128,n_fft=int(sr*0.5), hop_length=int(sr*1)
                                    )
                                ).transpose())
                fragment = fragment[np.newaxis,:]

                if one_data==[]:
                    one_data = fragment.copy()                    
                else:
                    one_data = np.vstack([one_data,fragment.copy()])
            
            # self repliction
            for i in range(15):
                if test_data==[]:
                    test_data = one_data.copy()                    
                else:
                    test_data = np.vstack([one_data,test_data.copy()])                    
                    
            # crnn test
            test_data = test_data.reshape(1,60,128,1)
            predict_va = model.predict(test_data)
            print(sum(sum(predict_va[0])/60),sum(sum(predict_va[1])/60))